In [6]:
import pandas as pd
import urllib
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

In [5]:
comments = pd.read_csv('../data/raw/train.csv')
comments.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [109]:
#train_comments = comments.query("split=='train'")
#test_comments = comments.query("split=='test'")

X = comments.iloc[:, 1:2]
y = comments.iloc[:, 2:]

Xtrain, Xvalidate, ytrain, yvalidate = train_test_split(X, y, test_size=0.3, random_state=1234)

In [110]:
Xtrain['comment_text']

57268     The sentence is OK for Latin America and the C...
90475     Notability \n\nI'm adding a notability templat...
50019     Further reading \n\nFurther reading contains s...
71600     BBC One ident images - fair use \n\nMany thank...
118447    Demographics?\nAre those figures like a flat i...
104555    I'm sure a collective term has been invented a...
12959     "\n\nRRTF Name Change Alert!\n  \nHi Name here...
84906     "\nCEDEM is the Electoral Commission...? And i...
114939    . Becuase of WP:Crystal, and the website never...
38978     "\n  This IP address has been blocked because ...
49722     Merge all character articles together onto a l...
33473     "\n\nHere's some - presumably - utterly ""acce...
139145    "\n\n == deflem said ""fuck you Wikipedia"" ==...
34471     "\nNot done, the above is highly promotional a...
90437     "\n\n Captions on Opentask \n\nHello.  I just ...
44197     I think G13 is only for Articles for Creation ...
142542    Semi-protected edit request on

In [28]:
# clf = Pipeline([
#     ('vect', CountVectorizer(max_features = 10000, ngram_range = (1,2))),
#     ('tfidf', TfidfTransformer(norm = 'l2')),
#     ('clf', LogisticRegression()),
# ])

In [121]:
#testing = Xtrain.values

#testing[0]

vect = CountVectorizer(max_features = 10000, ngram_range = (1,2))
data = vect.fit_transform(Xtrain['comment_text'])

data.shape


# clf = clf.fit(Xtrain, ytrain)

#auc = roc_auc_score(Xvalidate, clf.predict_proba(test_comments['comment'])[:, 1])
#print('Test ROC AUC: %.3f' %auc)

(111699, 10000)

In [122]:
tfidf = TfidfTransformer(norm = 'l2')
data_tfidf = tfidf.fit_transform(data)

data_tfidf.shape

(111699, 10000)

In [123]:
data_tfidf

<111699x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 6506530 stored elements in Compressed Sparse Row format>

In [124]:
clf = LogisticRegression()

clf.fit(data_tfidf, ytrain.iloc[:, 0])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [141]:
vect = CountVectorizer(max_features = 10000, ngram_range = (1,2))
data_validate = vect.fit_transform(Xvalidate['comment_text'])
data_validate_tfidf = tfidf.fit_transform(data_validate)

data_validate_tfidf.shape

# clf.score()

(47872, 10000)

In [138]:
# data_tfidf.todense()

matrix([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [142]:
clf.score(data_validate_tfidf, yvalidate.iloc[:, 0])

0.90514288101604279

In [144]:
auc = roc_auc_score(yvalidate.iloc[:, 1], clf.predict_proba(data_validate_tfidf)[:, 1])
print('Test ROC AUC: %.3f' %auc)

Test ROC AUC: 0.810
